# 02 | Experimentos

In [ ]:
import math, subprocess
import pandas as pd
import numpy as np
import os
from pathlib import Path
from IPython.display import display, clear_output
import statistics
from random import seed
from random import randint

In [ ]:
THREADS = 30
result_dir = "archivos"
cantidadArchivos = 250
cantidadPalabrasPorArchivo = 100
ITERACIONES = 10

In [ ]:
def generarArchivoBalanceado(nombreArchivo, largoPalabra, k):
    os.makedirs(F"../data/balanceado/", exist_ok=True)
    pathArchivo = "../data/balanceado/" + nombreArchivo
    archivo = open(pathArchivo, "w")
    for l in range(97, 122 + 1):
        for i in range(k):
            sal = chr(l)
            for lp in np.random.randint(97, 122, largoPalabra - 1):
                sal += chr(lp)
            archivo.write(sal + "\n")
    return pathArchivo

def generarCarpetaBalanceado( cantArchivos, cantidadLetras, cantidadPalabraPorLetra ):
    filename = "balanceado"
    for i in range(cantArchivos):
        nombre = filename + str(i)
        generarArchivoBalanceado( nombre, cantidadLetras,cantidadPalabraPorLetra )
    

In [ ]:
def generarArchivoUniforme(nombreArchivo, largoPalabra, cantidadPalabras):
    os.makedirs(F"../data/uniforme/", exist_ok=True)
    pathArchivo = "../data/uniforme/" + nombreArchivo
    archivo = open(pathArchivo, "w")
    sal = ""
    largoPalabra = largoPalabra
    for i in range(cantidadPalabras):
        for lp in range(largoPalabra):
            sal += chr(randint(97, 122))
        archivo.write(sal + "\n")
        sal = ""
    return pathArchivo

def generarCarpetaUniformes(cantArchivos, cantLetras, cantidadPalabras):
    fileName = "uniforme"
    for i in range(cantArchivos):
        nombre = fileName + str(i)
        generarArchivoUniforme(nombre, cantLetras, cantidadPalabras)

In [ ]:
def generarArchivoNormal(nombreArchivo, largoPalabra, cantidadPalabras, _sigma):
    # a b c d e f g h i j k l m 13 n o p q r s t u v w x y z
    mu= 109.5 #normal( 109.5, 3 ) 
    sigma = _sigma
    #con estos valores, la probabilidad de generar un valor fuera del rago de los caracteres ascii es demasiado baja.
    #de todas formas si se generara alguno, lo descartamos antes de convertirlo a char
    os.makedirs(F"../data/normal/", exist_ok=True)
    pathArchivo = "../data/normal/" + nombreArchivo
    archivo = open(pathArchivo, "w")
    sal = ""
    largoPalabra = largoPalabra
    for i in range(cantidadPalabras):
        for lp in range(largoPalabra):
            val = int(np.random.normal()*sigma + mu)
            if val >=97 and val <= 122:
                sal += chr(val)
        archivo.write(sal + "\n")
        sal = ""
    archivo.close()
    return pathArchivo

def generarCarpetaNormal(cantArchivos, cantLetras, cantidadPalabras, sigma):
    fileName = "normal"
    for i in range(cantArchivos):
        nombre = fileName + str(i)
        generarArchivoNormal(nombre, cantLetras, cantidadPalabras, sigma)

In [ ]:
def correr_experimento(funcion, cant_threads, archivo_entrada):
    # Crear proceso para ejecutar el codigo.
    print(funcion, archivo_entrada)
    process = subprocess.Popen(["../build/archivos", cant_threads, os.path.abspath(archivo_entrada)], stderr=subprocess.PIPE, stdout=subprocess.PIPE, universal_newlines = True)

    # Correr experimento.
    exit_code = process.wait()

    # Verificar que el proceso no fallo.
    if exit_code != 0:
        print(exit_code)
        print(process.stderr.read())
        process.stderr.close()
        raise(F"Hubo un error en la experimentacion: {funcion} con la instancia {archivo_entrada}.")
    # Leer salida de STDERR con los tiempos de ejecucion de cada metodo.
    # [cant threads , cantidad archivos, tiempo promedio, maximoasignado, minimo asignado, media restante]
    resultado = process.stdout.readline()
    process.stdout.close()
    process.stderr.close()
       
    return resultado.split(" ")

In [ ]:
def suma(s,t):
    res = [0.0]*len(s)
    for i in range( len(s) ):
        res [i] = float(s[i]) + float(t[i])
    return res

In [ ]:
def promedio(s, repe):
    #display(s)
    #display(repe)
    res = [0.0]*len(s)
    for i in range(len(s)):
        res[i] = s[i] / int(repe)
    #display(res)
    return res

In [ ]:
dir_paths = "../data/uniforme/"
entries = []
for f in os.listdir("../data/uniforme/"):
    os.remove("../data/uniforme/"+ str(f) ) 

entries = []

carpeta = "uniforme"

dir_paths = "../data/uniforme/"

generarCarpetaUniformes( cantidadArchivos, 6, cantidadPalabrasPorArchivo )

for t in range( 1, THREADS+1 ):
    clear_output(wait=True)
    display(F"Ejecutando carpetas {carpeta} con cantidad de Threads {t} " )
    salida_temp = [0.0]*8
    for i in range (ITERACIONES):
        salida = correr_experimento("CargarArchivos", str(t), dir_paths)
        salida_temp = suma( salida_temp, salida )
        #display(salida_temp)
    #display( salida )
    #display(salida_temp)
    salida_temp = promedio(salida_temp, ITERACIONES)
    #salida_temp = [ str(s) for s in salida_temp ]
    #display(salida_temp)
    entries.append(salida_temp)
#display( entries )
df_resultado = pd.DataFrame(entries, columns=["cantThreads", "cantArchivos", "tiempoMedio", "medLoad"])
os.makedirs(F"results/{result_dir}/", exist_ok=True)

#clear_output(wait=True)
display(F"Guardando los resultados en el archivo results/{result_dir}/uniforme.csv")
df_resultado.to_csv(F"results/{result_dir}/uniforme.csv", index=False, header=True)


In [ ]:
entries = []
for f in os.listdir("../data/uniforme/"):
    os.remove("../data/uniforme/"+ str(f) ) 

entries = []

carpeta = "normal"

dir_paths = "../data/normal/"

generarCarpetaNormal(cantidadArchivos, 6, cantidadPalabrasPorArchivo, 3)

for t in range( 1, THREADS+1 ):
    clear_output(wait=True)
    display(F"Ejecutando carpetas {carpeta} con cantidad de Threads {t} " )
    salida_temp = [0.0]*8
    for i in range(ITERACIONES):
        salida = correr_experimento("CargarArchivos", str(t), dir_paths)
        salida_temp = suma(salida_temp, salida)
    #display( salida )
    salida_temp = promedio(salida_temp, ITERACIONES)
    entries.append(salida_temp)
#display( entries )
df_resultado = pd.DataFrame(entries, columns=["cantThreads", "cantArchivos", "tiempoMedio", "medLoad"])
os.makedirs(F"results/{result_dir}/", exist_ok=True)

clear_output(wait=True)
display(F"Guardando los resultados en el archivo results/{result_dir}/normalsigma3.csv")
df_resultado.to_csv(F"results/{result_dir}/normalsigma3.csv", index=False, header=True)

In [ ]:
sigmas = ["0.125","0.25", "0.375", "0.5","0.75","0.875", "1", "1.25", "1.5", "1.75", "2","2.25", "2.5","2.75", "3","3.25", "3.5", "4", "4.5", "5", "6", "7", "8"]

entries = []

for f in os.listdir("../data/normal/"):
    os.remove("../data/normal/"+ str(f) )

dir_normal = "../data/normal/"


for i in sigmas:
    generarCarpetaNormal(cantidadArchivos, 10, cantidadPalabrasPorArchivo, float(i) )
    clear_output(wait=True)
    display( F"Sigma = {i}" )
    salida_temp = [0.0]*8
    for j in range(ITERACIONES):
        salida = correr_experimento("CargarArchivos", str(4), dir_normal)
        salida_temp = suma(salida_temp, salida)
    salida_temp = promedio(salida_temp, ITERACIONES)
    salida_temp.append(i)
    entries.append(salida_temp)
    df_resultado = pd.DataFrame(entries, columns=["cantThreads", "cantArchivos", "tiempoMedio", "medLoad", "sigma"])
    os.makedirs( F"results/{result_dir}/", exist_ok=True )

clear_output(wait=True)
display(F"Guardando los resultados en el archivo results/{result_dir}/sigma.csv")
df_resultado.to_csv(F"results/{result_dir}/sigma.csv", index=False, header=True)

In [ ]:
sigmas = ["0.125", "0.25", "0.5", "1", "2", "3", "4", "8"]

entries = []
#eliminamos archivos viejosw
for f in  os.listdir( "../data/normal/" ):
    os.remove("../data/normal/" + str(f))

dir_normal = "../data/normal/"

for sigma in sigmas:
    entries = []
    for i in range(100, 1001, 100):
        generarCarpetaNormal( 100, 10, i, float(sigma) )
        clear_output(wait=True)
        salida_temp = [0.0]*8
        for j in range(3):
            salida = correr_experimento("CargarArchivos", str(4), dir_normal)
            salida_temp = suma(salida_temp, salida)
        salida_temp = promedio(salida_temp, ITERACIONES)
        salida_temp.append(i)
        entries.append(salida_temp)
        df_resultado = pd.DataFrame(entries, columns=["cantThreads", "cantArchivos", "tiempoMedio", "medLoad", "cantPalabras"])
        os.makedirs( F"results/{result_dir}/", exist_ok=True )
    clear_output(wait=True)
    display(F"Guardando los resultados en el archivo results/{result_dir}/sigma{sigma}variandocantPalabras.csv")
    df_resultado.to_csv(F"results/{result_dir}/sigma{sigma}variandocantPalabras.csv", index=False, header=True)

